In [5]:
import random
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import GroupKFold
import pandas as pd
import math
import time
import os
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph, Namespace

ModuleNotFoundError: No module named 'sklearn'

# Get the features and gold standard 

In [ ]:
drugfeatfiles = ['drugs-fingerprint-sim.csv','drugs-se-sim.csv', 
                     'drugs-ppi-sim.csv', 'drugs-target-go-sim.csv','drugs-target-seq-sim.csv']
diseasefeatfiles =['diseases-hpo-sim.csv',  'diseases-pheno-sim.csv' ]
feature_folder ="data/features"



In [ ]:
drugfeatfiles = [ os.path.join(feature_folder, fn) for fn in drugfeatfiles]
diseasefeatfiles = [ os.path.join(feature_folder, fn) for fn in diseasefeatfiles]

In [ ]:
drugfeatfiles



In [ ]:
diseasefeatfiles


In [ ]:
drug_ind="data/input/openpredict-omim-drug.csv"

In [ ]:
#goldindfile = 'data/input/openpredict-omim-drug.csv'
drugDiseaseKnown = pd.read_csv(drug_ind,delimiter=',') 
drugDiseaseKnown.head()

In [ ]:
drugDiseaseKnown.rename(columns={'drugid':'Drug','omimid':'Disease'}, inplace=True)
drugDiseaseKnown.Disease = drugDiseaseKnown.Disease.astype(str)
drugDiseaseKnown.head()

# Merge feature matrix

In [9]:
def adjcencydict2matrix(df, name1, name2):
    df1 = df.copy()
    df1= df1.rename(index=str, columns={name1: name2, name2: name1})
    print (len(df))
    df =df.append(df1)
    print (len(df))
    return df.pivot(index=name1, columns=name2)

def mergeFeatureMatrix(drugfeatfiles, diseasefeatfiles):
    for i,featureFilename in enumerate(drugfeatfiles):
        print (featureFilename)
        df = pd.read_csv(featureFilename, delimiter=',')
        cond = df.Drug1 > df.Drug2
        df.loc[cond, ['Drug1', 'Drug2']] = df.loc[cond, ['Drug2', 'Drug1']].values
        if i != 0:
            drug_df=drug_df.merge(df,on=['Drug1','Drug2'],how='inner')
            #drug_df=drug_df.merge(temp,how='outer',on='Drug')
        else:
            drug_df =df
    drug_df.fillna(0, inplace=True)
    
    drug_df = adjcencydict2matrix(drug_df, 'Drug1', 'Drug2')
    drug_df = drug_df.fillna(1.0)

    
    for i,featureFilename in enumerate(diseasefeatfiles):
        print (featureFilename)
        df=pd.read_csv(featureFilename, delimiter=',')
        cond = df.Disease1 > df.Disease2
        df.loc[cond, ['Disease1','Disease2']] = df.loc[cond, ['Disease2','Disease1']].values
        if i != 0:
            disease_df = disease_df.merge(df,on=['Disease1','Disease2'], how='inner')
            #drug_df=drug_df.merge(temp,how='outer',on='Drug')
        else:
            disease_df = df
    disease_df.fillna(0, inplace=True)
    disease_df.Disease1 = disease_df.Disease1.astype(str)
    disease_df.Disease2 = disease_df.Disease2.astype(str)
    
    disease_df = adjcencydict2matrix(disease_df, 'Disease1', 'Disease2')
    disease_df = disease_df.fillna(1.0)
    
    return drug_df, disease_df

In [10]:
drug_df, disease_df = mergeFeatureMatrix(drugfeatfiles, diseasefeatfiles)

data/features/drugs-fingerprint-sim.csv
data/features/drugs-se-sim.csv
data/features/drugs-ppi-sim.csv
data/features/drugs-target-go-sim.csv
data/features/drugs-target-seq-sim.csv
129795
259590
data/features/diseases-hpo-sim.csv
data/features/diseases-pheno-sim.csv
44850
89700


In [11]:
drug_df.head()

TC                                                              \
Drug2     DB00014   DB00035   DB00091   DB00104   DB00122   DB00125   DB00136   
Drug1                                                                           
DB00014  1.000000  0.732558  0.574713  0.655172  0.313953  0.361446  0.402299   
DB00035  0.732558  1.000000  0.544304  0.697368  0.240506  0.441176  0.289157   
DB00091  0.574713  0.544304  1.000000  0.539474  0.365079  0.303030  0.397059   
DB00104  0.655172  0.697368  0.539474  1.000000  0.256757  0.347826  0.378378   
DB00122  0.313953  0.240506  0.365079  0.256757  1.000000  0.291667  0.267857   

                                       ... TARGETSEQ-SIM                      \
Drug2     DB00153   DB00162   DB00165  ...       DB01576   DB01577   DB01579   
Drug1                                  ...                                     
DB00014  0.321839  0.244186  0.376471  ...      0.076672  0.148647  0.039052   
DB00035  0.219512  0.150000  0.378378  ...      0.024000  0.030028  0.023312   
DB00091  0.294118  0.250000  0.384615  ...      0.039643  0.045992  0.036710   
DB00104  0.283784  0.208333  0.469697  ...      0.178035  0.180648  0.048190   
DB00122  0.280000  0.400000  0.200000  ...      0.040534  0.040534  0.041460   

                                                                               
Drug2     DB01590   DB01623   DB02546   DB04272   DB04844   DB04861   DB06287  
Drug1                                                                          
DB00014  0.028215  0.081568  0.043257  0.053699  0.053920  0.113543  0.028215  
DB00035  0.043115  0.023471  0.033298  0.031706  0.024000  0.021341  0.043115  
DB00091  0.016369  0.031936  0.059601  0.073739  0.039643  0.047390  0.016369  
DB00104  0.015979  0.172655  0.040032  0.052726  0.054976  0.188184  0.019203  
DB00122  0.027075  0.033040  0.050565  0.065972  0.036566  0.042961  0.035405  

[5 rows x 3060 columns]

In [12]:
disease_df.head()

HPO-SIM                                                    \
Disease2    102100    102300    102400    102500    103100    103780   
Disease1                                                               
102100    1.000000  0.228439  0.420190  0.432394  0.365865  0.091080   
102300    0.228439  1.000000  0.291239  0.257371  0.480403  0.427043   
102400    0.420190  0.291239  1.000000  0.464588  0.334854  0.083732   
102500    0.432394  0.257371  0.464588  1.000000  0.266952  0.221382   
103100    0.365865  0.480403  0.334854  0.266952  1.000000  0.322904   

                                                  ... PHENO-SIM                \
Disease2    104130    104300    106400    107320  ...    607682 607685 607850   
Disease1                                          ...                           
102100    0.312120  0.210254  0.516519  0.341580  ...       0.0    0.0    0.0   
102300    0.425632  0.510889  0.326918  0.204134  ...       0.0    0.0    0.0   
102400    0.257330  0.273490  0.484795  0.210790  ...       0.0    0.0    0.0   
102500    0.427080  0.263057  0.439328  0.184396  ...       0.0    0.0    0.0   
103100    0.426191  0.459806  0.370533  0.453242  ...       0.0    0.0    0.0   

                                                           
Disease2 608033 608088 608105 608217 608320 608583 608622  
Disease1                                                   
102100      0.0    0.0    0.0    0.0    0.0    0.0    0.0  
102300      0.0    0.0    0.0    0.0    0.0    0.0    0.0  
102400      0.0    0.0    0.0    0.0    0.0    0.0    0.0  
102500      0.0    0.0    0.0    0.0    0.0    0.0    0.0  
103100      0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[5 rows x 600 columns]

# Generate positive and negative pairs

In [13]:
def generatePairs(drug_df, disease_df, drugDiseaseKnown):
    drugwithfeatures = set(drug_df.columns.levels[1])
    diseaseswithfeatures = set(disease_df.columns.levels[1])
    
    drugDiseaseDict  = set([tuple(x) for x in  drugDiseaseKnown[['Drug','Disease']].values])

    commonDrugs= drugwithfeatures.intersection( drugDiseaseKnown.Drug.unique())
    commonDiseases=  diseaseswithfeatures.intersection(drugDiseaseKnown.Disease.unique() )
    print ("commonDrugs: %d commonDiseases : %d"%(len(commonDrugs),len(commonDiseases)))

    #abridged_drug_disease = [(dr,di)  for  (dr,di)  in drugDiseaseDict if dr in drugwithfeatures and di in diseaseswithfeatures ]

    #commonDrugs = set( [ dr  for dr,di in  abridged_drug_disease])
    #commonDiseases  =set([ di  for dr,di in  abridged_drug_disease])

    print ("Gold standard, associations: %d drugs: %d diseases: %d"%(len(drugDiseaseKnown),len(drugDiseaseKnown.Drug.unique()),len(drugDiseaseKnown.Disease.unique())))
    print ("Drugs with features: %d Diseases with features: %d"%(len(drugwithfeatures),len(diseaseswithfeatures)))
    print ("commonDrugs: %d commonDiseases : %d"%(len(commonDrugs),len(commonDiseases)))

    pairs=[]
    classes=[]
    for dr in commonDrugs:
        for di in commonDiseases:
            cls = (1 if (dr,di) in drugDiseaseDict else 0)
            pairs.append((dr,di))
            classes.append(cls)
            
    return pairs, classes

In [14]:
pairs, classes = generatePairs(drug_df, disease_df, drugDiseaseKnown)

commonDrugs: 510 commonDiseases : 300
Gold standard, associations: 1933 drugs: 592 diseases: 313
Drugs with features: 510 Diseases with features: 300
commonDrugs: 510 commonDiseases : 300


# Balance negative samples/postives 

In [15]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
def balance_data(pairs, classes, n_proportion):
    classes = np.array(classes)
    pairs = np.array(pairs)
    
    indices_true = np.where(classes == 1)[0]
    indices_false = np.where(classes == 0)[0]

    np.random.shuffle(indices_false)
    indices = indices_false[:(n_proportion*indices_true.shape[0])]
    print ("+/-:", len(indices_true), len(indices), len(indices_false))
    pairs = np.concatenate((pairs[indices_true], pairs[indices]), axis=0)
    classes = np.concatenate((classes[indices_true], classes[indices]), axis=0) 
    
 
    return pairs, classes

In [16]:
n_proportion = 2
pairs, classes= balance_data(pairs, classes, n_proportion)

+/-: 1671 3342 151329


# Train-Test Splitting

In [17]:
pairs_train, pairs_test, classes_train, classes_test = model_selection.train_test_split(pairs, classes, stratify=classes, test_size=0.2, shuffle=True)

In [18]:
len(pairs_train), len(pairs_test)

(4010, 1003)

# Feature extraction (Best Combined similarity)

In [19]:
def geometricMean(drug, disease, knownDrugDisease, drugDF, diseaseDF):
    #print (drug, disease)
    a  = drugDF.loc[knownDrugDisease[:,0]][drug].values
    b  = diseaseDF.loc[knownDrugDisease[:,1]][disease].values
    #print (a,b)
    c = np.sqrt( np.multiply(a,b) )
    ix2 = (knownDrugDisease == [drug, disease])
    c[ix2[:,1]& ix2[:,0]]=0.0
    return float(np.max(c))


def createFeatureDF(pairs, classes, knownDrugDisease, drugDFs, diseaseDFs):
    totalNumFeatures = len(drugDFs)*len(diseaseDFs)
    #featureMatri x= np.empty((len(classes),totalNumFeatures), float)
    df =pd.DataFrame(list(zip(pairs[:,0], pairs[:,1], classes)), columns =['Drug','Disease','Class'])
    index = 0
    for i,drug_col in enumerate(drugDFs.columns.levels[0]):
        for j,disease_col in enumerate(diseaseDFs.columns.levels[0]):
            drugDF = drugDFs[drug_col]
            diseaseDF = diseaseDFs[disease_col]
            df["Feature_"+str(drug_col)+'_'+str(disease_col)] = df.apply(lambda row: geometricMean( row.Drug, row.Disease, knownDrugDisease, drugDF, diseaseDF), axis=1)
    return df

def calculateCombinedSimilarity(pairs_train, pairs_test, classes_train, classes_test, drug_df, disease_df, knownDrugDisease):
    train_df  = createFeatureDF(pairs_train, classes_train, knownDrugDisease, drug_df, disease_df)
    test_df = createFeatureDF(pairs_test, classes_test, knownDrugDisease, drug_df, disease_df)
    return train_df, test_df

In [20]:
knownDrugDisease= pairs_train[classes_train==1]
train_df, test_df = calculateCombinedSimilarity(pairs_train, pairs_test, classes_train, classes_test, drug_df, disease_df, knownDrugDisease)

# Model Training

In [48]:
from sklearn import tree, ensemble
from sklearn import svm, linear_model, neighbors

def trainModel(train_df, clf):
    #features = list(train_df.columns.difference(['Drug','Disease','Class']))
    features= ['Feature_GO-SIM_HPO-SIM',
 'Feature_GO-SIM_PHENO-SIM',
 'Feature_PPI-SIM_HPO-SIM',
 'Feature_PPI-SIM_PHENO-SIM',
 'Feature_SE-SIM_HPO-SIM',
 'Feature_SE-SIM_PHENO-SIM',
 'Feature_TARGETSEQ-SIM_HPO-SIM',
 'Feature_TARGETSEQ-SIM_PHENO-SIM',
 'Feature_TC_HPO-SIM',
 'Feature_TC_PHENO-SIM']
    X = train_df[features]
    y = train_df['Class']
    X.head()
    print ('fiting classifier...')
    clf.fit(X, y)
    return clf

In [37]:
features = list(train_df.columns.difference(['Drug','Disease','Class']))
X = train_df[features]
y = train_df['Class']

import h5py  
Xh5File = "Xfromdf.h5"
yH5file = "Yfromdf.h5"
X.to_hdf(Xh5File, "X")
y.to_hdf(yh5File, "y")


X.head()
y.head()

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [45]:
#features = list(train_df.columns.difference(['Drug','Disease','Class','0_HPO-SIM','0_PHENO-SIM']))
features= ['Feature_GO-SIM_HPO-SIM',
 'Feature_GO-SIM_PHENO-SIM',
 'Feature_PPI-SIM_HPO-SIM',
 'Feature_PPI-SIM_PHENO-SIM',
 'Feature_SE-SIM_HPO-SIM',
 'Feature_SE-SIM_PHENO-SIM',
 'Feature_TARGETSEQ-SIM_HPO-SIM',
 'Feature_TARGETSEQ-SIM_PHENO-SIM',
 'Feature_TC_HPO-SIM',
 'Feature_TC_PHENO-SIM']


X = train_df[features]
y = train_df['Class']

X.to_csv("Xdata.csv")
y.to_csv("ydata.csv")
features

['Feature_GO-SIM_HPO-SIM',
 'Feature_GO-SIM_PHENO-SIM',
 'Feature_PPI-SIM_HPO-SIM',
 'Feature_PPI-SIM_PHENO-SIM',
 'Feature_SE-SIM_HPO-SIM',
 'Feature_SE-SIM_PHENO-SIM',
 'Feature_TARGETSEQ-SIM_HPO-SIM',
 'Feature_TARGETSEQ-SIM_PHENO-SIM',
 'Feature_TC_HPO-SIM',
 'Feature_TC_PHENO-SIM']

In [39]:

features

['Feature_GO-SIM_HPO-SIM',
 'Feature_GO-SIM_PHENO-SIM',
 'Feature_PPI-SIM_HPO-SIM',
 'Feature_PPI-SIM_PHENO-SIM',
 'Feature_SE-SIM_HPO-SIM',
 'Feature_SE-SIM_PHENO-SIM',
 'Feature_TARGETSEQ-SIM_HPO-SIM',
 'Feature_TARGETSEQ-SIM_PHENO-SIM',
 'Feature_TC_HPO-SIM',
 'Feature_TC_PHENO-SIM']

In [46]:
X.isnull().values.any()
X.isnull().sum()



Feature_GO-SIM_HPO-SIM             0
Feature_GO-SIM_PHENO-SIM           0
Feature_PPI-SIM_HPO-SIM            0
Feature_PPI-SIM_PHENO-SIM          0
Feature_SE-SIM_HPO-SIM             0
Feature_SE-SIM_PHENO-SIM           0
Feature_TARGETSEQ-SIM_HPO-SIM      0
Feature_TARGETSEQ-SIM_PHENO-SIM    0
Feature_TC_HPO-SIM                 0
Feature_TC_PHENO-SIM               0
dtype: int64

In [30]:
y.isnull().sum()

0

In [49]:
n_seed = 100
clf = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, random_state=n_seed) 
clfx = trainModel(train_df, clf)

fiting classifier...


# Evaulation 

In [57]:
from sklearn import metrics
import numbers
def multimetric_score(estimator, X_test, y_test, scorers):
    """Return a dict of score for multimetric scoring"""
    scores = {}
    for name, scorer in scorers.items():
        if y_test is None:
            score = scorer(estimator, X_test)
        else:
            score = scorer(estimator, X_test, y_test)

        if hasattr(score, 'item'):
            try:
                # e.g. unwrap memmapped scalars
                score = score.item()
            except ValueError:
                # non-scalar?
                pass
        scores[name] = score

        if not isinstance(score, numbers.Number):
            raise ValueError("scoring must return a number, got %s (%s) "
                             "instead. (scorer=%s)"
                             % (str(score), type(score), name))
    return scores

def evaluate(test_df, clf):
    features = list(train_df.columns.difference(['Drug','Disease','Class']))
    X_test =  test_df[features]
    y_test = test_df['Class']

    scoring = ['precision', 'recall', 'accuracy', 'roc_auc', 'f1', 'average_precision']
    scorers, multimetric = metrics.scorer._check_multimetric_scoring(clf, scoring=scoring)
    scores = multimetric_score(clf, X_test, y_test, scorers)
    return scores

In [58]:
scores = evaluate(test_df, clf)
print ("Test:",scores)

AttributeError: module 'sklearn.metrics' has no attribute 'scorer'

# 10-fold drug-disjoint cross-validation (PREDICT - CV scheme )

In [ ]:
disjoint = True
n_fold = 10

if disjoint:
    print ('Disjoint')
    groups = pairs[:,0] # group by drug
    group_kfold = GroupKFold(n_splits=n_fold)
    cv = group_kfold.split(pairs, classes, groups)
else:
    print ('Non-disjoint')
    skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=n_seed)
    cv = skf.split(pairs, classes)

n_seed = 100
cv_results = pd.DataFrame()
clf = linear_model.LogisticRegression(penalty='l2', solver='lbfgs', dual=False, tol=0.0001, C=1.0, random_state=n_seed) 
  
for i, (train, test) in enumerate(cv):
    print ('Fold',i+1)
    start_time = time.time()
    pairs_train = pairs[train]
    classes_train = classes[train] 
    pairs_test = pairs[test]
    classes_test = classes[test]
    knownDrugDisease= pairs_train[classes_train==1]
    
    train_df, test_df = calculateCombinedSimilarity(pairs_train, pairs_test, classes_train, classes_test, drug_df, disease_df, knownDrugDisease)
    elapsed_time = time.time() - start_time
    print ('Time elapsed to generate features:',time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

    clf = trainModel(train_df, clf)
    
    scores = evaluate(test_df, clf)
    #print ("Scores:",scores)
    cv_results = cv_results.append(scores, ignore_index=True)

In [ ]:
cv_results.mean()

In [ ]:
os.mkdir('results')
cv_results.to_csv('results/disjoint_lr.csv')

In [ ]:
#import pandas as pd
#cv_results = pd.read_csv('results/disjoint_lr.csv')

In [ ]:
cv_results.head()

In [ ]:
import time
def generateURI(prefix):
    uniqueID= int(round(time.time() * 1000))
    uri = URIRef(prefix+str(uniqueID))
    return uri

In [ ]:
DC = Namespace("http://purl.org/dc/terms/")
MLS = Namespace("http://www.w3.org/ns/mls#")
RPC = Namespace("https://w3id.org/reproduceme#")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

g =  ConjunctiveGraph(identifier = URIRef('http://bio2rdf.org/openpredict_resource:bio2rdf.dataset.openpredict.R1')) 

#graphURI = URIRef('http://bio2rdf.org/openpredict_resource:bio2rdf.output.openpredict.R1')
runURI = generateURI('http://www.w3.org/ns/mls#Run')
evalURI = generateURI('http://www.w3.org/ns/mls#ModelEvaluation')
evalSpecURI = generateURI('http://www.w3.org/ns/mls#EvaluationSpecification')

g.add((runURI, RDF['type'], MLS['Run']))
g.add((runURI, MLS['achieves'], RPC['Pipeline_OpenPREDICT']))

g.add((runURI, MLS['hasOutput'], evalURI))
g.add((evalURI, RDF['type'], MLS['ModelEvaluation']))

g.add((evalSpecURI, MLS['defines'],  RPC['Pipeline_OpenPREDICT']))
g.add((evalSpecURI, MLS['hasPart'],  MLS['TenFoldCrossValidation']))
g.add((MLS['TenFoldCrossValidation'], RDF['type'],  MLS['EvaluationProcedure']))
g.add((MLS['TenFoldCrossValidation'], RDFS['label'],  Literal('10-fold CV')))

g.add((evalSpecURI, MLS['hasPart'],  MLS['DrugwiseCrossValidation']))
g.add((MLS['DrugwiseCrossValidation'], RDF['type'],  MLS['EvaluationProcedure']))
g.add((MLS['DrugwiseCrossValidation'], RDFS['label'],  Literal('Drugwise CrossValidation')))
g.add((MLS['DrugwiseCrossValidation'], DC['description'],  Literal('Split drugs in 10-fold, remove drugs of each fold in the gold standard and consequently remove all the known indication sassociated with them')))

for index, value in cv_results.mean().items():
    measureURI = generateURI('http://www.w3.org/ns/mls#Measure_'+index)
    g.add((evalSpecURI, MLS['hasPart'],  measureURI))      
    g.add((evalURI, MLS['specifiedBy'],measureURI ))
    g.add((measureURI, RDF['type'], MLS['EvaluationMeasure']))
    g.add((measureURI, RDFS['label'],  Literal(index)))
    g.add((measureURI, MLS['hasValue'],  Literal(value)))



In [ ]:
outfile ='results/results_disjoint_lr.nq'
g.serialize(outfile, format='nquads')
print('RDF is generated at '+outfile)

In [ ]:
#import shutil
#import os
#shutil.copytree('results',os.path.join(abs_path, 'data/results/'))